In [1]:
import pandas as pd

In [2]:
df_sini = pd.read_feather("../../../data/interim/siniestros.feather")

In [3]:
df_ids = pd.read_feather("../../../data/interim/poliza_x_cliente.feather")

Verificación de cruce

In [4]:
len(set(df_ids["NUM_SECU_POL"]).intersection(set(df_sini["NUM_SECU_POL"]))) / len(df_ids)

0.007383039728453833

In [5]:
len(set(df_sini["NUM_SECU_POL"]).intersection(set(df_ids["NUM_SECU_POL"]))) / len(df_sini)

0.3896858920205821

### Agregamos CIF_ID a los Siniestros

In [6]:
df_sini_id = pd.merge(df_sini, df_ids, on=['NUM_SECU_POL'], how='left')

In [7]:
df_sini_id.drop_duplicates(subset=["NUM_SECU_POL", "FEC_DENU_SINI"], keep='last', inplace=True)

### Agrupamos siniestros por periodo

In [8]:
data = {fecha:i + 1 for i,fecha in enumerate(sorted(df_sini_id["FEC_DENU_SINI"].unique()))}

In [9]:
data

{'2018-01': 1,
 '2018-02': 2,
 '2018-03': 3,
 '2018-04': 4,
 '2018-05': 5,
 '2018-06': 6,
 '2018-07': 7,
 '2018-08': 8,
 '2018-09': 9,
 '2018-10': 10,
 '2018-11': 11,
 '2018-12': 12,
 '2019-01': 13,
 '2019-02': 14,
 '2019-03': 15,
 '2019-04': 16}

In [10]:
df_sini_id["periodo_denu_sini"] = df_sini_id["FEC_DENU_SINI"].map(data)

In [11]:
df_sini_id["periodo_liquidacion_sini"] = df_sini_id["FECHA_LIQUIDACION"].map(data)

In [12]:
df_sini_id["periodo_rechazo_sini"] = df_sini_id["FECHA_RECHAZO"].map(data)

In [13]:
to_pivot = df_sini_id[["CIF_ID","NUM_SECU_POL","TIPO_SINIESTRO","periodo_denu_sini","periodo_liquidacion_sini","periodo_rechazo_sini"]]

### Pivots por CIF_ID

In [14]:
pd.set_option("display.max_columns", 500)

In [49]:
df_sini = to_pivot.pivot_table(index='CIF_ID',columns=['periodo_denu_sini'], values=['NUM_SECU_POL','periodo_liquidacion_sini','periodo_rechazo_sini'],aggfunc='count', fill_value=0)

In [50]:
df_sini.head()

NUM_SECU_POL                                               \
periodo_denu_sini           1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16   
CIF_ID                                                                        
1559.0                       0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1   
5096.0                       0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0   
5575.0                       0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0   
7870.0                       0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0   
8495.0                       0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0   

                  periodo_liquidacion_sini                                   \
periodo_denu_sini                       1  2  3  4  5  6  7  8  9  10 11 12   
CIF_ID                                                                        
1559.0                                   0  0  0  0  0  0  0  0  0  0  0  0   
5096.0                                   0  0  0  0  0  0  0  0  0  0  0  0   
5575.0                                   0  0  0  0  0  0  0  0  0  0  0  0   
7870.0                                   0  0  0  0  0  0  0  0  0  0  0  0   
8495.0                                   0  0  0  0  0  0  0  0  0  0  0  0   

                              periodo_rechazo_sini                             \
periodo_denu_sini 13 14 15 16                   1  2  3  4  5  6  7  8  9  10   
CIF_ID                                                                          
1559.0             0  0  0  0                    0  0  0  0  0  0  0  0  0  0   
5096.0             0  0  0  0                    0  0  0  0  0  0  0  0  0  0   
5575.0             0  0  0  0                    0  0  0  0  0  0  0  0  0  0   
7870.0             0  0  0  0                    0  0  0  0  0  0  0  0  0  0   
8495.0             0  0  0  0                    0  0  0  0  0  0  0  0  0  0   

                                     
periodo_denu_sini 11 12 13 14 15 16  
CIF_ID                               
1559.0             0  0  0  0  0  0  
5096.0             0  0  0  0  0  0  
5575.0             0  0  0  0  0  0  
7870.0             0  0  0  0  0  0  
8495.0             0  0  0  0  0  0

In [32]:
df_tipo = to_pivot.pivot_table(index='CIF_ID',columns=['periodo_denu_sini','TIPO_SINIESTRO'],aggfunc='size', fill_value=0)

In [33]:
df_tipo.head()

periodo_denu_sini 1              2              3              4               \
TIPO_SINIESTRO     A  B  C  I  N  A  B  C  I  N  A  B  C  I  N  A  B  C  I  N   
CIF_ID                                                                          
1559.0             0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
5096.0             0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0   
5575.0             0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0   
7870.0             0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
8495.0             0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   

periodo_denu_sini 5                 6              7              8            \
TIPO_SINIESTRO     A  B  C  I  L  N  A  B  C  I  N  A  B  C  I  N  A  B  C  I   
CIF_ID                                                                          
1559.0             0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
5096.0             0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
5575.0             0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
7870.0             0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
8495.0             0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   

periodo_denu_sini    9              10                11                       \
TIPO_SINIESTRO     N  A  B  C  I  N  A  B  C  I  L  N  A  B  C  E  H  I  N  Z   
CIF_ID                                                                          
1559.0             0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
5096.0             0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
5575.0             0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
7870.0             0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
8495.0             0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   

periodo_denu_sini 12                   13                         14           \
TIPO_SINIESTRO     A  B  C  H  I  L  N  A  B  C  E  H  I  L  N  R  A  B  C  E   
CIF_ID                                                                          
1559.0             0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
5096.0             0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
5575.0             0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
7870.0             0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
8495.0             0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0   

periodo_denu_sini                15                            16              \
TIPO_SINIESTRO     H  I  L  N  Z  A  B  C  E  H  I  L  N  R  Z  A  B  C  E  H   
CIF_ID                                                                          
1559.0             0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
5096.0             0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
5575.0             0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
7870.0             0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0   
8495.0             0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   

periodo_denu_sini                 
TIPO_SINIESTRO     I  L  N  R  Z  
CIF_ID                            
1559.0             0  0  1  0  0  
5096.0             0  0  0  0  0  
5575.0             0  0  0  0  0  
7870.0             0  0  0  0  0  
8495.0             0  0  0  0  0

Reagrupamos los DF

In [57]:
df_sini = pd.DataFrame(df_sini.to_records())
df_tipo = pd.DataFrame(df_tipo.to_records())

In [58]:
df_sini.head()

,CIF_ID,"('NUM_SECU_POL', 1)","('NUM_SECU_POL', 2)","('NUM_SECU_POL', 3)","('NUM_SECU_POL', 4)","('NUM_SECU_POL', 5)","('NUM_SECU_POL', 6)","('NUM_SECU_POL', 7)","('NUM_SECU_POL', 8)","('NUM_SECU_POL', 9)","('NUM_SECU_POL', 10)","('NUM_SECU_POL', 11)","('NUM_SECU_POL', 12)","('NUM_SECU_POL', 13)","('NUM_SECU_POL', 14)","('NUM_SECU_POL', 15)","('NUM_SECU_POL', 16)","('periodo_liquidacion_sini', 1)","('periodo_liquidacion_sini', 2)","('periodo_liquidacion_sini', 3)","('periodo_liquidacion_sini', 4)","('periodo_liquidacion_sini', 5)","('periodo_liquidacion_sini', 6)","('periodo_liquidacion_sini', 7)","('periodo_liquidacion_sini', 8)","('periodo_liquidacion_sini', 9)","('periodo_liquidacion_sini', 10)","('periodo_liquidacion_sini', 11)","('periodo_liquidacion_sini', 12)","('periodo_liquidacion_sini', 13)","('periodo_liquidacion_sini', 14)","('periodo_liquidacion_sini', 15)","('periodo_liquidacion_sini', 16)","('periodo_rechazo_sini', 1)","('periodo_rechazo_sini', 2)","('periodo_rechazo_sini', 3)","('periodo_rechazo_sini', 4)","('periodo_rechazo_sini', 5)","('periodo_rechazo_sini', 6)","('periodo_rechazo_sini', 7)","('periodo_rechazo_sini', 8)","('periodo_rechazo_sini', 9)","('periodo_rechazo_sini', 10)","('periodo_rechazo_sini', 11)","('periodo_rechazo_sini', 12)","('periodo_rechazo_sini', 13)","('periodo_rechazo_sini', 14)","('periodo_rechazo_sini', 15)","('periodo_rechazo_sini', 16)"
0,1559.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,5096.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,5575.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,7870.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,8495.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [37]:
df_tipo.head()

,CIF_ID,"(1, 'A')","(1, 'B')","(1, 'C')","(1, 'I')","(1, 'N')","(2, 'A')","(2, 'B')","(2, 'C')","(2, 'I')","(2, 'N')","(3, 'A')","(3, 'B')","(3, 'C')","(3, 'I')","(3, 'N')","(4, 'A')","(4, 'B')","(4, 'C')","(4, 'I')","(4, 'N')","(5, 'A')","(5, 'B')","(5, 'C')","(5, 'I')","(5, 'L')","(5, 'N')","(6, 'A')","(6, 'B')","(6, 'C')","(6, 'I')","(6, 'N')","(7, 'A')","(7, 'B')","(7, 'C')","(7, 'I')","(7, 'N')","(8, 'A')","(8, 'B')","(8, 'C')","(8, 'I')","(8, 'N')","(9, 'A')","(9, 'B')","(9, 'C')","(9, 'I')","(9, 'N')","(10, 'A')","(10, 'B')","(10, 'C')","(10, 'I')","(10, 'L')","(10, 'N')","(11, 'A')","(11, 'B')","(11, 'C')","(11, 'E')","(11, 'H')","(11, 'I')","(11, 'N')","(11, 'Z')","(12, 'A')","(12, 'B')","(12, 'C')","(12, 'H')","(12, 'I')","(12, 'L')","(12, 'N')","(13, 'A')","(13, 'B')","(13, 'C')","(13, 'E')","(13, 'H')","(13, 'I')","(13, 'L')","(13, 'N')","(13, 'R')","(14, 'A')","(14, 'B')","(14, 'C')","(14, 'E')","(14, 'H')","(14, 'I')","(14, 'L')","(14, 'N')","(14, 'Z')","(15, 'A')","(15, 'B')","(15, 'C')","(15, 'E')","(15, 'H')","(15, 'I')","(15, 'L')","(15, 'N')","(15, 'R')","(15, 'Z')","(16, 'A')","(16, 'B')","(16, 'C')","(16, 'E')","(16, 'H')","(16, 'I')","(16, 'L')","(16, 'N')","(16, 'R')","(16, 'Z')"
0,1559.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,5096.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,5575.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,7870.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,8495.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [51]:
len(df_tipo) == len(df_sini)

True

** Cambio de nombre de columnas **

In [52]:
df_tipo.columns

Index(['CIF_ID', '(1, 'A')', '(1, 'B')', '(1, 'C')', '(1, 'I')', '(1, 'N')',
       '(2, 'A')', '(2, 'B')', '(2, 'C')', '(2, 'I')',
       ...
       '(16, 'A')', '(16, 'B')', '(16, 'C')', '(16, 'E')', '(16, 'H')',
       '(16, 'I')', '(16, 'L')', '(16, 'N')', '(16, 'R')', '(16, 'Z')'],
      dtype='object', length=106)

In [62]:
df_tipo = df_tipo.rename(columns=lambda x: x.replace("(","").replace(")","").replace(", ","_TIPO_SINI_").replace("'",""))

In [63]:
df_tipo.head()

,index,CIF_ID,1_TIPO_SINI_A,1_TIPO_SINI_B,1_TIPO_SINI_C,1_TIPO_SINI_I,1_TIPO_SINI_N,2_TIPO_SINI_A,2_TIPO_SINI_B,2_TIPO_SINI_C,2_TIPO_SINI_I,2_TIPO_SINI_N,3_TIPO_SINI_A,3_TIPO_SINI_B,3_TIPO_SINI_C,3_TIPO_SINI_I,3_TIPO_SINI_N,4_TIPO_SINI_A,4_TIPO_SINI_B,4_TIPO_SINI_C,4_TIPO_SINI_I,4_TIPO_SINI_N,5_TIPO_SINI_A,5_TIPO_SINI_B,5_TIPO_SINI_C,5_TIPO_SINI_I,5_TIPO_SINI_L,5_TIPO_SINI_N,6_TIPO_SINI_A,6_TIPO_SINI_B,6_TIPO_SINI_C,6_TIPO_SINI_I,6_TIPO_SINI_N,7_TIPO_SINI_A,7_TIPO_SINI_B,7_TIPO_SINI_C,7_TIPO_SINI_I,7_TIPO_SINI_N,8_TIPO_SINI_A,8_TIPO_SINI_B,8_TIPO_SINI_C,8_TIPO_SINI_I,8_TIPO_SINI_N,9_TIPO_SINI_A,9_TIPO_SINI_B,9_TIPO_SINI_C,9_TIPO_SINI_I,9_TIPO_SINI_N,10_TIPO_SINI_A,10_TIPO_SINI_B,10_TIPO_SINI_C,10_TIPO_SINI_I,10_TIPO_SINI_L,10_TIPO_SINI_N,11_TIPO_SINI_A,11_TIPO_SINI_B,11_TIPO_SINI_C,11_TIPO_SINI_E,11_TIPO_SINI_H,11_TIPO_SINI_I,11_TIPO_SINI_N,11_TIPO_SINI_Z,12_TIPO_SINI_A,12_TIPO_SINI_B,12_TIPO_SINI_C,12_TIPO_SINI_H,12_TIPO_SINI_I,12_TIPO_SINI_L,12_TIPO_SINI_N,13_TIPO_SINI_A,13_TIPO_SINI_B,13_TIPO_SINI_C,13_TIPO_SINI_E,13_TIPO_SINI_H,13_TIPO_SINI_I,13_TIPO_SINI_L,13_TIPO_SINI_N,13_TIPO_SINI_R,14_TIPO_SINI_A,14_TIPO_SINI_B,14_TIPO_SINI_C,14_TIPO_SINI_E,14_TIPO_SINI_H,14_TIPO_SINI_I,14_TIPO_SINI_L,14_TIPO_SINI_N,14_TIPO_SINI_Z,15_TIPO_SINI_A,15_TIPO_SINI_B,15_TIPO_SINI_C,15_TIPO_SINI_E,15_TIPO_SINI_H,15_TIPO_SINI_I,15_TIPO_SINI_L,15_TIPO_SINI_N,15_TIPO_SINI_R,15_TIPO_SINI_Z,16_TIPO_SINI_A,16_TIPO_SINI_B,16_TIPO_SINI_C,16_TIPO_SINI_E,16_TIPO_SINI_H,16_TIPO_SINI_I,16_TIPO_SINI_L,16_TIPO_SINI_N,16_TIPO_SINI_R,16_TIPO_SINI_Z
0,0,1559.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1,5096.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,5575.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,7870.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,8495.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [59]:
df_sini.columns

Index(['CIF_ID', '('NUM_SECU_POL', 1)', '('NUM_SECU_POL', 2)',
       '('NUM_SECU_POL', 3)', '('NUM_SECU_POL', 4)', '('NUM_SECU_POL', 5)',
       '('NUM_SECU_POL', 6)', '('NUM_SECU_POL', 7)', '('NUM_SECU_POL', 8)',
       '('NUM_SECU_POL', 9)', '('NUM_SECU_POL', 10)', '('NUM_SECU_POL', 11)',
       '('NUM_SECU_POL', 12)', '('NUM_SECU_POL', 13)', '('NUM_SECU_POL', 14)',
       '('NUM_SECU_POL', 15)', '('NUM_SECU_POL', 16)',
       '('periodo_liquidacion_sini', 1)', '('periodo_liquidacion_sini', 2)',
       '('periodo_liquidacion_sini', 3)', '('periodo_liquidacion_sini', 4)',
       '('periodo_liquidacion_sini', 5)', '('periodo_liquidacion_sini', 6)',
       '('periodo_liquidacion_sini', 7)', '('periodo_liquidacion_sini', 8)',
       '('periodo_liquidacion_sini', 9)', '('periodo_liquidacion_sini', 10)',
       '('periodo_liquidacion_sini', 11)', '('periodo_liquidacion_sini', 12)',
       '('periodo_liquidacion_sini', 13)', '('periodo_liquidacion_sini', 14)',
       '('periodo_liquidacion

In [60]:
df_sini = df_sini.rename(columns=lambda x: x.replace("(","").replace(")","").replace(", ","_").replace("'","").replace("NUM_SECU_POL","periodo_sini"))

In [61]:
df_sini.columns

Index(['CIF_ID', 'periodo_sini_1', 'periodo_sini_2', 'periodo_sini_3',
       'periodo_sini_4', 'periodo_sini_5', 'periodo_sini_6', 'periodo_sini_7',
       'periodo_sini_8', 'periodo_sini_9', 'periodo_sini_10',
       'periodo_sini_11', 'periodo_sini_12', 'periodo_sini_13',
       'periodo_sini_14', 'periodo_sini_15', 'periodo_sini_16',
       'periodo_liquidacion_sini_1', 'periodo_liquidacion_sini_2',
       'periodo_liquidacion_sini_3', 'periodo_liquidacion_sini_4',
       'periodo_liquidacion_sini_5', 'periodo_liquidacion_sini_6',
       'periodo_liquidacion_sini_7', 'periodo_liquidacion_sini_8',
       'periodo_liquidacion_sini_9', 'periodo_liquidacion_sini_10',
       'periodo_liquidacion_sini_11', 'periodo_liquidacion_sini_12',
       'periodo_liquidacion_sini_13', 'periodo_liquidacion_sini_14',
       'periodo_liquidacion_sini_15', 'periodo_liquidacion_sini_16',
       'periodo_rechazo_sini_1', 'periodo_rechazo_sini_2',
       'periodo_rechazo_sini_3', 'periodo_rechazo_sini_4',

### Ahora unimos los dataframes

In [65]:
df = pd.merge(df_sini, df_tipo, on=['CIF_ID'], how='outer')

In [67]:
len(df_sini.columns) + len(df_tipo.columns)

156

In [69]:
len(df.columns)

155

In [70]:
df.to_feather("../../../data/processed/siniestros_x_cif_id.feather")